In [5]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding,LSTM, CuDNNLSTM, Flatten, MaxPooling1D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, TimeDistributed, Conv1D, Conv2D
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import h5py
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import shuffle
from time import time

import os
import sys

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

NameError: name 'os' is not defined

In [3]:
from label_transforms import *

ModuleNotFoundError: No module named 'label_transforms'

In [7]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()
from sagemaker import get_execution_role

role = get_execution_role()
bucket='files-and-examples-01'
train_file = 'datasets/su_dataset/video_data_low_level.hdf5'

train_path = s3.open('s3://{}/{}'.format(bucket, train_file))

f = h5py.File(name=train_path, mode='r')
x = np.array(f['x'])
y = np.array(f['y'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [11]:
X_train.shape

(7743, 1549, 17)

In [12]:
seq_len=1549
STEPS = 50
EPOCHS= 60

model = Sequential()

model.add(LSTM(units=100,
              return_sequences=True,
              input_shape=(seq_len,17)))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(100, activation='relu'))

model.add(Dense(44, activation='sigmoid'))


model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
model.summary()

Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-10-13 12:56:34.033 tensorflow-2-6-gpu--ml-g4dn-xlarge-6f7b193ebf85c4ca6ed374f84b1e:21 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-13 12:56:34.060 tensorflow-2-6-gpu--ml-g4dn-xlarge-6f7b193ebf85c4ca6ed374f84b1e:21 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1549, 100)         47200     
_________________________________________________________________
dropout (Dropout)         

In [ ]:
model.fit(X_train, y_train, batch_size=50, epochs=EPOCHS, validation_data=(X_test, y_test))

Epoch 1/60
155/155 [==============================] - 742s 5s/step - loss: 3.6040 - accuracy: 0.0576 - val_loss: 3.5270 - val_accuracy: 0.0684
Epoch 2/60
155/155 [==============================] - 740s 5s/step - loss: 3.3083 - accuracy: 0.1099 - val_loss: 3.4715 - val_accuracy: 0.0798
Epoch 3/60
155/155 [==============================] - 740s 5s/step - loss: 3.0851 - accuracy: 0.1534 - val_loss: 3.5468 - val_accuracy: 0.0801
Epoch 4/60
155/155 [==============================] - 742s 5s/step - loss: 2.8288 - accuracy: 0.2194 - val_loss: 3.6869 - val_accuracy: 0.0913
Epoch 5/60
155/155 [==============================] - 741s 5s/step - loss: 2.5278 - accuracy: 0.2960 - val_loss: 3.9187 - val_accuracy: 0.0865
Epoch 6/60
155/155 [==============================] - 739s 5s/step - loss: 2.2096 - accuracy: 0.3793 - val_loss: 4.2118 - val_accuracy: 0.0922
Epoch 7/60
155/155 [==============================] - 739s 5s/step - loss: 1.9263 - accuracy: 0.4577 - val_loss: 4.5726 - val_accuracy: 0.0826

# Training on basic emotions

In [2]:
ang_idx = emotion_abr_to_emotion_id["ang"]
print("ang_idx:", ang_idx)
hap_idx = emotion_abr_to_emotion_id["hap"]
print("hap_idx:", hap_idx)
sad_idx = emotion_abr_to_emotion_id["sad"]
print("sad:", sad_idx)
disg_idx = emotion_abr_to_emotion_id["disg"]
print("disg:", disg_idx)
fea_idx = emotion_abr_to_emotion_id["fea"]
print("fea:", fea_idx)

NameError: name 'emotion_abr_to_emotion_id' is not defined

In [ ]:
y_basic_emotions[y_basic_emotions == ang_idx] = 0
y_basic_emotions[y_basic_emotions == hap_idx] = 1
y_basic_emotions[y_basic_emotions == sad_idx] = 2
y_basic_emotions[y_basic_emotions == disg_idx] = 3
y_basic_emotions[y_basic_emotions == fea_idx] = 4